# Creativity Evaluation Dataset Preparation

Create New Model Instance For the testing

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import pandas as pd
from datasets import load_from_disk
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from langchain import PromptTemplate
from utils.evaluate import calculate_cosine_similarity
from tqdm import tqdm
import psutil
import subprocess
import signal
import re

In [ ]:
def find_uvicorn_processes():
    uvicorn_processes = []
    for process in psutil.process_iter(["pid", "name", "cmdline"]):
        if "uvicorn" in process.info["name"].lower() or (
            "uvicorn" in process.info["cmdline"] if process.info["cmdline"] else False
        ):
            uvicorn_processes.append(process.info)

    return uvicorn_processes


def kill_process_by_pid(pid):
    try:
        process = psutil.Process(pid)
        process.terminate()  # Alternatively, use process.kill() for a forceful termination
        print(f"Process {pid} has been terminated.")
    except psutil.NoSuchProcess:
        print(f"No process found with PID {pid}.")


uvicorn_processes = find_uvicorn_processes()

if uvicorn_processes:
    print("The follwoing is ruuning: ")
    for process in uvicorn_processes:
        print(
            f"PID: {process['pid']}, Name: {process['name']}, Command: {' '.join(process['cmdline'])}"
        )
else:
    print("Uvicore unfound")

In [ ]:
directory = "causal"
module = "text_generation_api_llama2_7B"
model = "LLama_7B"
host = "127.0.0.1"
port = 8899
quantization = "4bit"
python_executable = "~/Personal_Directories/srb/causalEnv/bin/python"
test_num = 300

In [ ]:
import subprocess
from concurrent.futures import ThreadPoolExecutor

# %nohup run_uvicorn_command(directory, your_module, host,port)
uvicorn_command = f"{python_executable} -m uvicorn {directory}.{module}:app --reload --host {host} --port {port}"
subprocess.Popen(uvicorn_command, shell=True)

In [42]:
from collections import Counter
import pandas as pd, json

with open(
    "/home/lujun/local/causalllm/causal/creativity/creativity_eval/stories/ttcw_short_stories.json",
    "r",
) as f:
    stories = json.load(f)

with open(
    "/home/lujun/local/causalllm/causal/creativity/creativity_eval/tests/ttcw_all_tests.json",
    "r",
) as f:
    tests = json.load(f)

with open(
    "/home/lujun/local/causalllm/causal/creativity/creativity_eval/annotations/ttcw_annotations.json",
    "r",
) as f:
    annotations = json.load(f)

# Each of the 14 tests was completed by 3 independent expert annotators.
# 48 stories x 14 tests x 3 annotators = 2016 annotations
print("Total of %d annotations" % len(annotations))


def full_prompt2context(full_prompt):
    # remove the last two non-empty lines
    lines = full_prompt.strip().split("\n")
    kept1 = "\n".join(lines[:-1]).strip().split("\n")
    kept2 = kept1[:-1]
    return "\n".join(kept2).strip()


for test in tests:
    test["expanded_context"] = full_prompt2context(test["full_prompt"])

Total of 2016 annotations


In [55]:
import pandas as pd

print(f"Length of Stories {len(stories)}")
new_yorker_stories = [story for story in stories if "GPT3.5" in story["story_id"]]
print(f"Length of new_yorker Stroies {len(new_yorker_stories)}")
df = pd.DataFrame(new_yorker_stories)
df.to_csv("/home/lujun/local/causalllm/resource/creativity/story_plot.csv", index=False)

Length of Stories 48
Length of new_yorker Stroies 12


# Create Dataset For The Model

In [56]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import pandas as pd
from datasets import load_from_disk
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from langchain import PromptTemplate
from utils.evaluate import calculate_cosine_similarity
from tqdm import tqdm
import psutil
import subprocess
import signal
import re
from utils import generate_story_by_loop

In [67]:
local_data_csv = "/home/lujun/local/causalllm/resource/creativity/story_plot.csv"
df = pd.read_csv(local_data_csv)
df["word_count"] = df["content"].apply(lambda x: len(x.split()))
df.head(1)

,story_idx,story_id,story_name,plot,content,word_count
0,0,0_GPT3.5,"Maintenance, Hvidovre",A woman experiences a disorienting night in a ...,The clock struck midnight as Lily stumbled thr...,1992


In [68]:
initial_prompt = """Write a New Yorker-style story given the plot below. Make sure it is at least {word_count} words.
Directly start with the story, do not say things like 'Here's the story [...]

Current plot: {plot}

"""

recurrent_check_prompt = """
You wrote the story I gave you below. I requested a story with {word_count} words, but the story only has {current_word_count} words.
Can you rewrite the story to make it longer, and closer to the {word_count} word target I gave you. 
Directly start with the story, do not say things like ‘Here’s the story [...]:

Current story: {story}

"""


def generate_story(plot, word_count):
    return generate_story_by_loop(
        plot,
        initial_prompt,
        recurrent_check_prompt,
        word_count,
        0.8,
        max_new_tokens=512,
        max_length=2048,
    )


df["story"] = df.apply(
    lambda row: generate_story(row["plot"], row["word_count"]), axis=1
)